In [ ]:
from livelossplot.tf_keras import PlotLossesCallback
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.metrics import confusion_matrix
from sklearn import metrics

import numpy as np
np.random.seed(42)
import warnings;warnings.simplefilter('ignore')
%matplotlib inline
print('Tensorflow version:', tf.__version__)

## Task 2: Load and Preprocess SETI Data

In [ ]:
train_images = pd.read_csv('dataset/train/images.csv', header=None)
train_labels = pd.read_csv('dataset/train/labels.csv', header=None)
eval_images = pd.read_csv('dataset/evaluation/images.csv', header=None)
eval_labels = pd.read_csv('dataset/evaluation/labels.csv', header=None)

In [ ]:
x_train = train_images.values.reshape(3200,64,128,1)
x_val = eval_images.values.reshape(800,64,128,1)

y_train = train_labels.values
y_val = eval_labels.values


## Task 3: Plot 2D Spectrograms

In [ ]:
plt.figure(0, figsize=(12,12))
for i in range(1,4):
    plt.subplot(1,3,i)
    img = np.squeeze(x_train[np.random.randint(0, x_train.shape[0])])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap='gray')

## Task 4: Create Training and Validation Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(horizontal_flip=True)
datagen_train.fit(x_train)
datagen_val = ImageDataGenerator(horizontal_flip=True)
datagen_val.fit(x_train)

## Task 5: Creating the CNN Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Initialising the CNN
model = Sequential( )
# 1st Convolution
model.add(Conv2D(32,(5,5), padding='same', input_shape=(64,128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# 2nd Convolution layer
model.add(Conv2D(64,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
# Fully connected layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(4,activation='softmax'))

## Task 6: Learning Rate Scheduling and Compile the Model


In [ ]:
initial_learning_rate = 0.005
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate = initial_learning_rate, decay_steps=5, decay_rate=0.96,staircase=True
)
optimizer = Adam(learning_rate=lr_scheduler)

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Task 7: Training the Model

In [ ]:
checkpoint = ModelCheckpoint('model_weight.h5', monitor='val_loss',save_weights_only=True, mode='min',verbose=0)
callbacks = [PlotLossesCallback(), checkpoint]

batch_size = 32
history = model.fit(
    datagen_train.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch = len(x_train) //batch_size,
    validation_data = datagen_val.flow(x_val, y_val, batch_size=batch_size, shuffle=True),
    validation_steps= len(x_val)//batch_size,
    epochs=12,
    callbacks=callbacks
)


## Task 8: Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns

y_true = np.argmax(y_val,1)
y_pred = np.argmax(model.predict(x_val),1)
print(metrics.classification_report(y_true,y_pred))

In [ ]:
print("Classification accuracy: %o.6f" % metrics.accuracy_score(y_true,y_pred))

In [ ]:
labels = ["squiggle", "narrowband", "noise", "narrowbanddrd"]

ax= plt.subplot()
sns.heatmap(metrics.confusion_matrix(y_true, y_pred, normalize='true'), annot=True, ax = ax, cmap=plt.cm.Blues); #annot=True to annotate cells

# labels, title and ticks
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels)